In [ ]:
import os
import gc
import ssl
import time
import numpy as np
import pandas as pd
import gudhi as gd

from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from persim.persistent_entropy import *

from utils import *

In [ ]:
pca = PCA(n_components=2)

ssl._create_default_https_context = ssl._create_unverified_context

housing = fetch_california_housing()

X_all = pd.DataFrame(housing.data, columns = housing.feature_names)
y_all = housing['target']

X_all = X_all#.iloc[:3000]
y_all = y_all#[:3000]

scaler = StandardScaler()
scaler.fit(X_all)
X_scaled = pd.DataFrame(scaler.transform(X_all), index=X_all.index, columns=X_all.columns)
X_scaled.head()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=43)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.7195934214967924
R-square: 0.6113250004234638


### Add Topology feature

In [14]:
remove_infinity = lambda alpha_list: [alpha for alpha in alpha_list if alpha[1] != np.inf]

def compute_persistence(data_remove):
    print(f"進程 {os.getpid()} 開始計算")
    
    # PCA
    pca_result = pca.fit_transform(data_remove)
    # Alpha complex
    alpha_complex = gd.AlphaComplex(points = pca_result)
    st_alpha = alpha_complex.create_simplex_tree()
    alpha_filtration = st_alpha.get_filtration()
    alpha_list = list(alpha_filtration)
    # Filter
    filtered_alpha_list = remove_infinity(alpha_list)
    dgm = np.array([[0.0, value] for _, value in filtered_alpha_list])
    dgm_filtered = np.array([bar for bar in dgm if bar[1] - bar[0] != 0])
    ## entropy
    PE = gd.representations.Entropy()
    pe_normal = PE.fit_transform([dgm_filtered])
    
    print(f"進程 {os.getpid()} 完成計算")
    return pe_normal

In [ ]:
data_list = Data_list(X_scaled)
len(data_list)

In [ ]:
results = Parrallel_compute(data_list, typeComplex='AlphafromCechmate')

平行運算時間:  63.30636692047119


In [ ]:
# Remove Infinity
remove_infinity = lambda alpha_list: [alpha for alpha in alpha_list if alpha[1] != np.inf]

entropy_feature = []

start = time.time()
for i in range(X_scaled.shape[0]):
    if (i + 1) % 100 == 0:
        print(f"Processing row {i + 1} out of {X_scaled.shape[0]}")
        gc.collect()
        
    data_remove = X_scaled.drop(index=i)
    
    # PCA
    pca_result = pca.fit_transform(data_remove)
    
    # Alpha complex
    alpha_complex = gd.AlphaComplex(points = pca_result)
    st_alpha = alpha_complex.create_simplex_tree()
    alpha_filtration = st_alpha.get_filtration()
    alpha_list = list(alpha_filtration)
    
    # Filter
    filtered_alpha_list = remove_infinity(alpha_list)
    dgm = np.array([[0.0, value] for _, value in filtered_alpha_list])
    dgm_filtered = np.array([bar for bar in dgm if bar[1] - bar[0] != 0])
    
    ## entropy
    PE = gd.representations.Entropy()
    pe_normal = PE.fit_transform([dgm_filtered])
    
    entropy_feature.append(pe_normal)
    
end = time.time()
print("ripser++ total time: ", end-start)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.5545262813691066
R-square: 0.6644448697913721
